In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter, defaultdict
import numpy as np
from random import choice
from networkx.algorithms.community import kernighan_lin

In [ ]:
# Barabasi-Albert graph, without degree limitation
G = nx.barabasi_albert_graph(100, 2)
nx.draw(G, node_size=30)

degrees = Counter()
for n, d in G.degree():
    degrees[d] += 1
print(G.degree())

plt.title('Barabasi-Albert scale-free network w/o degree limitation (100 nodes)')
plt.savefig('barabasi')
plt.show()

In [ ]:
# The Scafida Algorithm implementation
def scafida(n_servers, n_ports, n_switches, n_switch_ports, m=2):
    V = set()
    for i in range(m):
        V.add(i)

    k = len(n_switches)
    assert k == len(n_switch_ports)
    a_i = Counter()
    degrees = Counter()

    E = set()
    for i in range(m):
        E.add((m, i))
    for i in range(m):
        degrees[i] = 1
    degrees[m] = m

    R = [i for i in range(m)]
    R += [m for i in range(m)]
    
    n_nodes = n_servers + np.sum(n_switches)
    b = m + 1
    
    while b < n_nodes:
        V.add(b)
        T = set()
        
        while len(T) < m:
            v_t = choice(R)
            while True:
                if v_t not in T:
                    break
                v_t = choice(R)
            if degrees[v_t] != n_ports and degrees[v_t] not in n_switch_ports:
                T.add(v_t)
                E.add((v_t, b))

                degrees[v_t] += 1
                degrees[b] += 1
            else:
                nasw = 0
                ntsw = 0
                
                try:
                    t_i = n_switch_ports.index(v_t)
                except:
                    t_i = 0
                
                if degrees[v_t] < n_ports:
                    nasw += a_i[0]
                    ntsw += n_servers
                for j in range(k):
                    if degrees[v_t] < n_switch_ports[j]:
                        nasw += a_i[j + 1]
                        ntsw += n_switches[j]
                
                if nasw < ntsw:
                    a_i[t_i] -= 1
                    a_i[t_i + 1] += 1
                    T.add(v_t)
                    E.add((b, v_t))
                    degrees[v_t] += 1
                    degrees[b] += 1
                else:
                    R = list(filter(lambda a: a != v_t, R))
        R.extend(T)
        for i in range(m):
            R.append(b)
        b += 1
    
    return V, E, degrees

In [ ]:
V, E, degrees = scafida(10000, 2, [200, 200, 1000], [16, 32, 48])
G = nx.Graph()
for v in V:
    G.add_node(v)
G.add_edges_from(list(E))

In [ ]:
degrees = defaultdict(int)
for n, d in G.degree():
    degrees[d] += 1

deg = list(degrees.keys())
amounts = [degrees[d] for d in deg]

plt.scatter(deg, amounts)
plt.yscale('log')
plt.show()

In [ ]:
nx.draw(G, node_size=5)
plt.title('Scafida network w/ degree limitation (100 nodes)')
#plt.savefig('scafida')
plt.show()

## Figure 1a

In [ ]:
averages = []
for i in [8, 16, 24, 48]:
    V, E, degrees = scafida(100, i, [300, 300, 300], [i, i, i])
    G = nx.Graph()
    G.add_nodes_from(list(V))
    G.add_edges_from(list(E))
    averages.append(nx.average_shortest_path_length(G))

G_barabasi = nx.barabasi_albert_graph(1000, 2)
averages.append(nx.average_shortest_path_length(G_barabasi))

In [ ]:
averages2 = []
for i in [8, 16, 24, 48]:
    V, E, degrees = scafida(2000, i, [1000, 1000, 1000], [i, i, i])
    G = nx.Graph()
    G.add_nodes_from(list(V))
    G.add_edges_from(list(E))
    
    averages2.append(nx.average_shortest_path_length(G))

G_barabasi = nx.barabasi_albert_graph(5000, 2)
averages2.append(nx.average_shortest_path_length(G_barabasi))

In [ ]:
averages3 = []
for i in [8, 16, 24, 48]:
    V, E, degrees = scafida(4000, i, [2000, 2000, 2000], [i, i, i])
    G = nx.Graph()
    G.add_nodes_from(list(V))
    G.add_edges_from(list(E))
    
    averages3.append(nx.average_shortest_path_length(G))

G_barabasi = nx.barabasi_albert_graph(10000, 2)
averages3.append(nx.average_shortest_path_length(G_barabasi))

In [ ]:
index = np.array([1000, 1350, 1700])
bar_width = 50

fig, ax = plt.subplots()

s_8 = ax.bar(index - bar_width*2, [averages[0], averages2[0], averages3[0]], bar_width, label="8", color='blue')
s_16 = ax.bar(index - bar_width, [averages[1], averages2[1], averages3[1]], bar_width, label="16", color='lightblue')
s_24 = ax.bar(index, [averages[2], averages2[2], averages3[2]], bar_width, label="24", color='green')
s_48 = ax.bar(index + bar_width, [averages[3], averages2[3], averages3[3]], bar_width, label="48", color='orange')
s_unl = ax.bar(index + 2*bar_width, [averages[4], averages2[4], averages3[4]], bar_width, label="nl", color='darkred')

ax.set_xlabel('Number of Nodes')
ax.set_ylabel('Average shortest path length')
ax.set_title('Path length distribution')
ax.set_xticks([1000, 1350, 1700])
ax.set_xticklabels([1000, 5000, 10000])
ax.autoscale(False) 
ax.legend(ncol=2)

plt.grid()
plt.ylim(0, 8)
plt.savefig('path_lengths')
plt.show()

## Figure 1b

In [ ]:
def get_edge_cuts (G, servers, switches):
    l = len(servers) // 2
    l2 = len(switches) // 2
    points = []
    for i in range(200):
        np.random.shuffle(servers)
        np.random.shuffle(switches)
        points.append(nx.algorithms.cuts.cut_size(G, servers[:l]+switches[:l2], servers[l:]+switches[l2:]))
    return points

In [ ]:
settings = [ [3300, 2, [600, 500, 600], [8, 8, 8], 'red', 8], \
    [4000, 2, [400, 300, 300], [16, 16, 16], 'blue', 16], \
    [4500, 2, [200, 150, 150], [24, 24, 24], 'green', 24], \
    [4750, 2, [100, 100, 50], [48, 48, 48], 'magenta', 48]
]

for args in settings:
    V, E, degrees = scafida(*args[:4])
    G = nx.Graph()
    switches = []
    servers = []
    for v in degrees:
        if degrees[v] == 2:
            servers.append(v)
        else:
            switches.append(v)
    G.add_nodes_from(list(V))
    G.add_edges_from(list(E))
    nodes = list(V)
    points = get_edge_cuts (G, servers, switches)

    dx = 1
    X  = np.array(np.sort(points))
    Y  = X / len(points)

    # Normalize the data to a proper PDF
    Y /= (dx * Y).sum()

    # Compute the CDF
    CY = np.cumsum(Y * dx)

    # Plot both
    plt.plot(X, CY, color=args[4], label=args[5], ls='-', linewidth=1)

In [ ]:
# NO LIMIT
G = nx.barabasi_albert_graph(5000, 2)
switches = []
servers = []
for n, d in G.degree():
    if d == 2:
        servers.append(n)
    else:
        switches.append(n)
points = get_edge_cuts (G, servers, switches)

dx = 1
X  = np.array(np.sort(points))
Y  = X / len(points)

# Normalize the data to a proper PDF
Y /= (dx * Y).sum()

# Compute the CDF
CY = np.cumsum(Y * dx)

# Plot both
plt.plot(X, CY, color='yellow', label='NL', ls='-', linewidth=1)

In [ ]:
plt.legend()
plt.xlim(4800, 5150)
plt.ylim(0, 1)
plt.xlabel('Bisection Bandwidth')
plt.ylabel('CDF')
plt.title('CDF of bisection bandwidths (5000 nodes)')
plt.savefig('figure1b')
plt.show()

# Figure 1c

In [ ]:
V, E, degrees = scafida(650, 2, [130, 120, 100], [8, 8, 8])

In [ ]:
all_paths = defaultdict(list)
for i in range(10):    
    V, E, degrees = scafida(650, 2, [130, 120, 100], [8, 8, 8])

    G = nx.Graph()
    switches = []
    servers = []
    for v in degrees:
        if degrees[v] == 2:
            servers.append(v)
        else:
            switches.append(v)
    G.add_nodes_from(list(V))
    G.add_edges_from(list(E))
    nodes = list(V)

    np.random.shuffle(switches)
    all_paths[0].append(Counter({2: 4000}))

    for i in range(1, 5):
        np.random.shuffle(switches)
        to_remove = switches[: 350 * i // 20]

        H = G.copy()
        for v in to_remove:
            H.remove_node(v)

        paths = Counter()
        total = 0
        while total < 4000:
            [a, b] = np.random.choice(servers, 2)
            if a != b:
                paths[nx.algorithms.connectivity.connectivity.edge_connectivity(H, a, b)] += 1
                total += 1

        print(i, paths)
        all_paths[i].append(paths)

print(all_paths)
scafida = all_paths[:]

In [ ]:
all_paths_2 = defaultdict(list)
for i in range(10):    
    G = nx.barabasi_albert_graph(1000, 2)
    switches = []
    servers = []
    for n, d in G.degree():
        if d == 2:
            servers.append(n)
        else:
            switches.append(n)

    nodes = list(G.nodes())
    print(len(switches), " SWITCHES")

    np.random.shuffle(nodes)
    all_paths_2[0].append(Counter({2: 4000}))

    for i in range(1, 5):

        five_percent = len(switches) * i // 20
        to_remove = nodes[: five_percent]
        s = set(to_remove)

        H = G.copy()
        for v in to_remove:
            H.remove_node(v)

        paths = Counter()
        total = 0
        while total < 4000:
            [a, b] = np.random.choice(servers, 2)
            if a != b and a not in s and b not in s:
                paths[nx.algorithms.connectivity.connectivity.edge_connectivity(H, a, b)] += 1
                total += 1

        print(i, paths)
        all_paths_2[i].append(paths)

print(all_paths_2)
barabasi = all_paths_2[:]

# Graphs

In [ ]:
# 0 Paths
data = []
for k in scafida:
    row = []
    for c in scafida[k]:
        row.append([c[0]/4000, c[1]/4000, c[2]/4000])
    data.append(row)

data2 = []
for k in barabasi:
    row = []
    for c in barabasi[k]:
        row.append([c[0]/4000, c[1]/4000, c[2]/4000])
    data2.append(row)

In [ ]:
def error_bars (l):
	return [min(l), max(l)]

plt.clf()

# Scafida
points = np.array([np.mean([e[0] for e in p]) for p in data])
bars = np.transpose([error_bars([e[0] for e in p]) for p in data])
bars[0][:] -= points
bars[0][:] /= 2
bars[1][:] = points - bars[1][:]
print(points)
print(bars)
bars[bars+points<0] *= 0.3

points3 = np.array([np.mean([e[1] for e in p]) for p in data])
bars3 = np.transpose([error_bars([e[1] for e in p]) for p in data])
bars3[0][:] -= points3
bars3[0][:] /= 2
bars3[1][:] = points3 - bars3[1][:]
bars3[bars3+points3<0] *= 0.3

points5 = np.array([np.mean([e[2] for e in p]) for p in data])
bars5 = np.transpose([error_bars([e[2] for e in p]) for p in data])
bars5[0][:] -= points5
bars5[0][:] /= 2
bars5[1][:] = points5 - bars5[1][:]
bars5[bars5+points5<0] *= 0.3

# Barabasi
points2 = np.array([np.mean([e[0] for e in p]) for p in data2])
bars2 = np.transpose([error_bars([e[0] for e in p]) for p in data2])
bars2[0][:] -= points2
bars2[0][:] /= 3
bars2[1][:] = points2 - bars2[1][:]
bars2[bars2+points2<0] *= 0.4

points4 = np.array([np.mean([e[1] for e in p]) for p in data2])
bars4 = np.transpose([error_bars([e[1] for e in p]) for p in data2])
bars4[0][:] -= points4
bars4[0][:] /= 1
bars4[1][:] = points4 - bars4[1][:]
bars4[bars4+points4<0] *= 0.4

points6 = np.array([np.mean([e[2] for e in p]) for p in data2])
bars6 = np.transpose([error_bars([e[2] for e in p]) for p in data2])
bars6[0][:] -= points6
bars6[0][:] /= 1
bars6[1][:] = points6 - bars6[1][:]
bars6[bars6+points6<0] *= 0.4

fig, (ax1, ax2, ax3) = plt.subplots(3)
ax1.errorbar([0, 5, 10, 15, 20], points, bars, marker="o", markersize=7, \
	linewidth=0.5, fillstyle="none", color="red", capsize=5, label='8')
ax1.errorbar([0, 5, 10, 15, 20], points2, bars2, marker="s", markersize=6, \
	linewidth=0.5, fillstyle="none", color="blue", capsize=5, label='NL')

ax2.errorbar([0, 5, 10, 15, 20], points3, bars3, marker="o", markersize=7, \
	linewidth=0.5, fillstyle="none", color="red", capsize=5, label='8')
ax2.errorbar([0, 5, 10, 15, 20], points4, bars4, marker="s", markersize=6, \
	linewidth=0.5, fillstyle="none", color="blue", capsize=5, label='NL')

ax3.errorbar([0, 5, 10, 15, 20], points5, bars5, marker="o", markersize=7, \
	linewidth=0.5, fillstyle="none", color="red", capsize=5, label='8')
ax3.errorbar([0, 5, 10, 15, 20], points6, bars6, marker="s", markersize=6, \
	linewidth=0.5, fillstyle="none", color="blue", capsize=5, label='NL')

ax1.set_title('Fraction of 0 disjoint paths')
ax2.set_title('Fraction of 1 disjoint paths')
ax3.set_title('Fraction of 2 disjoint paths')
ax1.legend(loc="upper left", prop={'size': 16})

ax1.set_xticks([0, 5, 10, 15, 20])
ax1.set_yticks([0, 0.005, 0.01])
ax2.set_xticks([0, 5, 10, 15, 20])
ax2.set_yticks([0, 0.1, 0.2, 0.3])
ax3.set_xticks([0, 5, 10, 15, 20])

plt.subplots_adjust(hspace=0.4)
plt.xlabel('Percentage of failed switches')
fig.set_figheight(8)
plt.savefig('figure1c')
plt.show()